In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('task_svo.xlsx', sheet_name='main')

In [3]:
df.replace('-',np.NaN,inplace=True)

In [4]:
df.head(5)

,№,Дата,Наименование а/к,Код автакомпании,Номер рейса,Тип рейса,Флаг прилет/вылет,Направление,Кол-во пассажиров,Международный/ Внутрений
0,1,2021-01-25,а/к Полет,PL,526,Регулярный,D,Самара,110.0,Внутрений
1,2,2021-01-25,а/к Мечта,MC,34,Грузовой,D,Владивосток,NaN,Внутрений
2,3,2021-01-25,а/к Мечта,MC,46,Чартер,D,Минеральные воды,146.0,Внутрений
3,4,2021-01-25,а/к Полет,PL,507,Чартер,A,Токио,225.0,Международный
4,5,2021-01-25,а/к Полет,PL,517,Регулярный,A,Нью-Йорк,179.0,Международный


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   №                         50 non-null     int64         
 1   Дата                      50 non-null     datetime64[ns]
 2   Наименование а/к          50 non-null     object        
 3   Код автакомпании          50 non-null     object        
 4   Номер рейса               50 non-null     int64         
 5   Тип рейса                 50 non-null     object        
 6   Флаг прилет/вылет         50 non-null     object        
 7   Направление               50 non-null     object        
 8   Кол-во пассажиров         43 non-null     float64       
 9   Международный/ Внутрений  50 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 4.0+ KB


In [6]:
# Количество прилетных рейсов
flight_count = df[df['Флаг прилет/вылет'] == "A"].groupby('Наименование а/к')['Международный/ Внутрений'].count()

In [7]:
flight_count = pd.DataFrame(flight_count).reset_index()

In [8]:
#Количество международных рейсов
international = df[df['Международный/ Внутрений'] == "Международный"].groupby('Наименование а/к')['Международный/ Внутрений'].count()

In [9]:
international = pd.DataFrame(international).reset_index()

In [10]:
#Самый частолетаемый рейс
often = df.groupby(['Наименование а/к','Направление'],as_index=False)['Номер рейса']\
.count().sort_values(by='Номер рейса', ascending=False).iloc[0:3]

In [11]:
often = pd.DataFrame(often).reset_index()

In [12]:
#Номер рейса с направлением Самара
Samara_number = df[df['Направление'] == 'Самара'].drop_duplicates(subset=['Наименование а/к'])

In [13]:
Samara_number = Samara_number[['Наименование а/к','Номер рейса']]

In [14]:
#Среднее количество пассажиров по всем рейсам авиакомпании
mean_passangers = df.groupby('Наименование а/к')['Кол-во пассажиров'].mean()

In [15]:
mean_passangers = pd.DataFrame(mean_passangers).reset_index()

In [16]:
svo_rslt = flight_count.merge (international, how='left', \
                               left_on='Наименование а/к',right_on='Наименование а/к')

In [17]:
svo_rslt = svo_rslt.merge (often, how='left', \
                               left_on='Наименование а/к',right_on='Наименование а/к')

In [18]:
svo_rslt = svo_rslt.merge (Samara_number, how='left', \
                               left_on='Наименование а/к',right_on='Наименование а/к')

In [19]:
svo_rslt = svo_rslt.merge (mean_passangers, how='left', \
                               left_on='Наименование а/к',right_on='Наименование а/к')

In [43]:
columns_name = ['Авиакомпания',
                'Количество прилетных рейсов',
                'Количество международных рейсов',
                'Самый частолетаемый рейс',
                'Номер рейса с направлением Самара',
               'Среднее количество пассажиров по всем рейсам авиакомпании']

In [30]:
svo_rslt = svo_rslt.drop(columns=['Номер рейса_x'], axis=1)

In [44]:
svo_rslt.columns = columns_name

In [45]:
svo_rslt

,Авиакомпания,Количество прилетных рейсов,Количество международных рейсов,Самый частолетаемый рейс,Номер рейса с направлением Самара,Среднее количество пассажиров по всем рейсам авиакомпании
0,а/к Крылья,9,12,Алма-Ата,1026,189.692308
1,а/к Мечта,6,1,Владивосток,35,167.357143
2,а/к Полет,10,14,Нью-Йорк,526,155.500000
